In [5]:
import openmeteo_requests
import numpy as np
import requests_cache
import pandas as pd
from retry_requests import retry


In [6]:
data_frame_storage=pd.DataFrame(columns=['date','real_temperature_2m', 'latitude', "longitude"])
# Setup the Open-Meteo API client with cache and retry on error
for i in np.arange(43,50,2):
	for j in np.arange(-5,8,2):
		cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
		retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
		openmeteo = openmeteo_requests.Client(session = retry_session)

		# Make sure all required weather variables are listed here
		# The order of variables in hourly or daily is important to assign them correctly below
		url = "https://archive-api.open-meteo.com/v1/archive"
		params = {
			"latitude": i,
			"longitude": j,
			"start_date": "2023-01-01",
			"end_date": "2023-12-31",
			"hourly": "temperature_2m"
		}
		responses = openmeteo.weather_api(url, params=params)

		# Process first location. Add a for-loop for multiple locations or weather models
		response = responses[0]


		# Process hourly data. The order of variables needs to be the same as requested.
		hourly = response.Hourly()
		hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

		hourly_data = {"date": pd.date_range(
			start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
			end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
			freq = pd.Timedelta(seconds = hourly.Interval()),
			inclusive = "left"
		)}
		hourly_data["real_temperature_2m"] = hourly_temperature_2m

		hourly_dataframe = pd.DataFrame(data = hourly_data)
		hourly_dataframe.dropna(inplace=True)
		hourly_dataframe["latitude"] = [i for k in range(len(hourly_dataframe))]
		hourly_dataframe["longitude"] = [j for k in range(len(hourly_dataframe))]
		data_frame_storage = pd.concat([data_frame_storage, hourly_dataframe], axis=0)

nom='hourly_past_real_temperature_2m_2023.csv'
data_frame_storage.to_csv(nom)

C:\Users\tsang\AppData\Local\Temp\ipykernel_12572\3952952367.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_frame_storage = pd.concat([data_frame_storage, hourly_dataframe], axis=0)


                          date  temperature_2m
0    2023-01-01 00:00:00+00:00       15.938999
1    2023-01-01 01:00:00+00:00       15.188999
2    2023-01-01 02:00:00+00:00       14.839000
3    2023-01-01 03:00:00+00:00       14.438999
4    2023-01-01 04:00:00+00:00       13.988999
...                        ...             ...
8755 2023-12-31 19:00:00+00:00        8.011000
8756 2023-12-31 20:00:00+00:00        7.511000
8757 2023-12-31 21:00:00+00:00        7.411000
8758 2023-12-31 22:00:00+00:00        7.661000
8759 2023-12-31 23:00:00+00:00        7.561000

[8755 rows x 2 columns]
